<img src='justtemp (6).png' width='75%'>

In [2]:
import numpy as np

In [3]:
x = np.random.randn(6,2,16)
y = np.random.randn(6,2,16)

In [4]:
x[0], y[0]

(array([[ 1.36470437,  0.87200251, -1.18250177,  0.44407913,  1.43573175,
         -0.96392373,  0.80048372,  0.37646168,  0.50118034, -0.54184833,
          0.56504924, -1.08860538, -2.25951443,  0.61756878, -0.67466977,
         -1.01981043],
        [ 1.54401809, -0.00483111,  2.16767851, -0.57002522,  0.16628122,
         -0.45750552, -0.92246314, -1.56876512, -1.6414432 , -0.37762223,
          0.88806984, -0.37810634,  0.52033298,  0.74484429, -0.56735781,
         -0.27987345]]),
 array([[ 1.72887491, -0.30841473, -0.6640174 ,  0.98275548, -0.50554817,
         -0.71441   ,  0.82188782,  0.44255599,  0.31975046, -1.47244758,
         -1.04747443, -0.07611002,  1.79969344, -1.35069793, -0.36791814,
         -0.70841665],
        [ 1.4404599 ,  1.29851254, -0.96777917,  1.33310005,  1.51382273,
         -0.56265939, -0.58545693,  2.76027713,  1.65141094, -0.72207618,
          0.01303044,  0.53231669,  0.01865763, -0.03647782,  0.39357133,
         -1.07791898]]))

In [5]:
weightsQ = np.random.randn(2,16)
weightsK = np.random.randn(2,16)
weightsV = np.random.randn(2,16)

In [6]:
Q = np.matmul(weightsQ, x[:,:,:,None]).squeeze(3)
K = np.matmul(weightsK, x[:,:,:,None]).squeeze(3)
V = np.matmul(weightsV, x[:,:,:,None]).squeeze(3)
Q.shape, K.shape, V.shape

((6, 2, 2), (6, 2, 2), (6, 2, 2))

In [7]:
weightsQ.shape

(2, 16)

In [8]:
def softmax(x, byline=True):
    if byline:
        x = np.clip(x, 1e-8, 1 - 1e-8)
        for i in range(x.shape[0]):
            x[i] = np.exp(x[i]) / (np.sum(np.exp(x[i])))

        return x

    x = np.clip(x, 1e-8, 1 - 1e-8)

    for i in range(x.shape[0]):
        for j in range(x.shape[1]):

            x[i, j] = np.exp(x[i, j]) / (np.sum(np.exp(x[i, j])))

    return x

In [9]:
QK = softmax(np.matmul(Q, K.reshape(K.shape[0], K.shape[2], Q.shape[1])) / Q.shape[1])

QK.shape

(6, 2, 2)

In [10]:
res = np.matmul(QK, V)

res.shape

(6, 2, 2)

In [11]:
def attention(weightsQ, weightsK, weightsV, x, masked=False, encoded=None, back_prop=False):

    Q = np.matmul(weightsQ, x[:,:,:,None]).squeeze(3)

    x = encoded if encoded is not None else x

    K = np.matmul(weightsK, x[:,:,:,None]).squeeze(3)
    V = np.matmul(weightsV, x[:,:,:,None]).squeeze(3)

    QK = np.matmul(Q, K.reshape(K.shape[0], K.shape[2], K.shape[1])) / Q.shape[1]
    
    if masked:
        for i in range(1, QK.shape[2]):
            QK[:i, i] = -np.inf

    Qk = softmax(QK)

    res = np.matmul(Qk, V)

    if back_prop:
        return Qk, Q, K, V, QK

    return res

In [12]:
x.shape

(6, 2, 16)

In [13]:
def multihead(x, heads, weightsQ=None, weightsK=None, weightsV=None, weightO=None, masked=False, encoded=None):
    dk = int(x.shape[2]/heads)
    should_add = False

    if weightsQ is None:
        weightsQ = []
        weightsK = []
        weightsV = []

        should_add = True

    for i in range(heads):
            
        if should_add:
            weightsQ.append(np.random.randn(dk, x.shape[2]) * 0.01)
            weightsK.append(np.random.randn(dk, x.shape[2]) * 0.01)
            weightsV.append(np.random.randn(dk, x.shape[2]) * 0.01)

        if i == 0:
            res = attention(weightsQ[i], weightsK[i], weightsV[i], x, masked, encoded)
            continue

        res = np.concatenate((res, attention(weightsQ[i], weightsK[i], weightsV[i], x, masked)), axis=2)

    if weightO is None:
        weightO = np.random.randn(heads*dk, x.shape[2])

    result = np.matmul(weightO, res[:,:,:,None]).squeeze(axis=3)

    if should_add:
        return result, weightsQ, weightsK, weightsV, weightO, res
    
    return result, res

In [14]:
res,_,_,_,_,_ = multihead(x, 8)
res.shape

(6, 2, 16)

In [15]:
def positional_encoding(x):
    for pos in range(x.shape[1]):
        if pos%2 == 0:
            x[:, pos] += np.sin(pos/10000**(2*pos/2 / x.shape[1]))
        else:
            x[:, pos] += np.cos(pos/10000**(2*(pos/2 -1) / x.shape[1]))

    return x

In [16]:
x[0]

array([[ 1.36470437,  0.87200251, -1.18250177,  0.44407913,  1.43573175,
        -0.96392373,  0.80048372,  0.37646168,  0.50118034, -0.54184833,
         0.56504924, -1.08860538, -2.25951443,  0.61756878, -0.67466977,
        -1.01981043],
       [ 1.54401809, -0.00483111,  2.16767851, -0.57002522,  0.16628122,
        -0.45750552, -0.92246314, -1.56876512, -1.6414432 , -0.37762223,
         0.88806984, -0.37810634,  0.52033298,  0.74484429, -0.56735781,
        -0.27987345]])

In [17]:
positional_encoding(x)[0]

array([[ 1.36470437,  0.87200251, -1.18250177,  0.44407913,  1.43573175,
        -0.96392373,  0.80048372,  0.37646168,  0.50118034, -0.54184833,
         0.56504924, -1.08860538, -2.25951443,  0.61756878, -0.67466977,
        -1.01981043],
       [ 2.40633697,  0.85748776,  3.02999738,  0.29229365,  1.0286001 ,
         0.40481335, -0.06014426, -0.70644625, -0.77912433,  0.48469664,
         1.75038871,  0.48421253,  1.38265185,  1.60716316,  0.29496106,
         0.58244542]])

In [18]:
def norm(x, gamma, beta):
    return gamma*(x-np.mean(x, axis=(0,1)))/np.std(x, axis=(0,1)) + beta

In [19]:
np.mean(norm(x, 1, 0)[0][0]), np.std(norm(x, 1, 0)[0][0])

(-0.5396290562691443, 0.9247159537163941)

In [20]:
def relu(x):
    return np.maximum(0, x)

In [21]:
def encoder(x):
    x = positional_encoding(x)
    
    mha, weightsQ, weightsK, weightsV, weightO, middle = multihead(x, 8)
    x += mha


    gamma = np.random.randn(x.shape[2]) * 0.01
    beta = np.random.randn(1) * 0.01
    x = norm(x, gamma, beta)

    weight1 = np.random.randn(x.shape[2]*4, x.shape[2]) * 0.01
    bias1 = np.random.randn(2,x.shape[2]*4) * 0.01
    weight2 = np.random.randn(x.shape[2], x.shape[2]*4) * 0.01
    bias2 = np.random.randn(2,x.shape[2]) * 0.01

    Z1 = np.matmul(weight1, x[:,:,:,None]).squeeze(3) + bias1
    A1 = relu(Z1)
    Z2 = np.matmul(weight2, A1[:,:,:,None]).squeeze(3) + bias2

    return Z2

In [22]:
encoder(x).shape

(6, 2, 16)

In [23]:
def decoder(x, encoded):
    x = positional_encoding(x)

    mha, weightsQ, weightsK, weightsV, weightO, middle = multihead(x, 8, masked=True)
    x += mha

    gamma1 = np.random.randn(x.shape[2]) * 0.01
    beta1 = np.random.randn(1) * 0.01
    x = norm(x, gamma1, beta1)

    mha2, weightsQ2, weightsK2, weightsV2, weightO2, middle2 = multihead(x, 8, encoded=encoded)
    x += mha2

    gamma2 = np.random.randn(x.shape[2]) * 0.01
    beta2 = np.random.randn(1)     * 0.01
    x = norm(x, gamma1, beta1)

    weight1 = np.random.randn(x.shape[2]*4, x.shape[2]) * 0.01
    bias1 = np.random.randn(2,x.shape[2]*4) * 0.01
    weight2 = np.random.randn(x.shape[2], x.shape[2]*4) * 0.01
    bias2 = np.random.randn(2,x.shape[2]) * 0.01

    Z1 = np.matmul(weight1, x[:,:,:,None]).squeeze(3) + bias1
    A1 = relu(Z1)
    Z2 = np.matmul(weight2, A1[:,:,:,None]).squeeze(3) + bias2

    return Z2

In [24]:
decoder(y, encoder(x)).shape

(6, 2, 16)

In [25]:
def final(x):
    weight1 = np.random.randn(16, x.shape[2]) * 0.01
    bias1 = np.random.randn(2,16) * 0.01

    Z1 = np.matmul(weight1, x[:,:,:,None]).squeeze(3) + bias1
    A1 = softmax(Z1, False)

    return A1

In [26]:
np.sum(final(decoder(y, encoder(x)))[0,0])

1.0

## BackProp
<img style='width:75%' src='justtemp (7).png'/>

For Skip Connection $$\cfrac{\delta L}{\delta x} = \cfrac{\delta L}{\delta H}\cfrac{\delta H}{\delta x} = \cfrac{\delta L}{\delta H} ( \cfrac{\delta F}{\delta x} + 1 ) = \cfrac{\delta L}{\delta H}\cfrac{\delta F}{\delta x} + \cfrac{\delta L}{\delta H}$$

In [27]:
def mse(y, y_hat):
    return np.mean((y - y_hat)**2, axis=(2))

In [28]:
mse(final(decoder(y, encoder(x).reshape(x.shape))).reshape(y.shape), y)

array([[ 1.00093971, 10.0489294 ],
       [ 1.14874611,  7.11292786],
       [ 0.29201138,  7.48690935],
       [ 0.71747214,  6.77709729],
       [ 1.37684997,  6.04899903],
       [ 0.78773674,  5.5905806 ]])

In [29]:
def one_hot(y = [1, 2, 16]):

    one_hot_matrix = np.zeros((y))

    for i in range(y[0]):
        for j in range(y[1]):
            one_hot_matrix[i,j,np.random.randint(y[2])] = 1

    return one_hot_matrix

In [30]:
x_true = np.random.randn(1,2,16)
y_true = one_hot([1, 2, 16])
heads = 8
dk = int(x_true.shape[2]/heads)

x = positional_encoding(x_true)

mha, weightsQ, weightsK, weightsV, weightO, middle = multihead(x, heads)
# mha = x
x1 = x + mha

gamma = np.random.randn(x.shape[2]) * 0.01
beta = np.random.randn(1) * 0.01
x2 = norm(x1, gamma, beta)

# x2 = x1

weight1 = np.random.randn(x.shape[2]*4, x.shape[2]) * 0.01
bias1 = np.random.randn(x.shape[1],x.shape[2]*4) * 0.01
weight2 = np.random.randn(x.shape[2], x.shape[2]*4) * 0.01
bias2 = np.random.randn(x.shape[1],x.shape[2]) * 0.01

Z1 = np.matmul(weight1, x2[:,:,:,None]).squeeze(3) + bias1
A1 = relu(Z1)
encoded = np.matmul(weight2, A1[:,:,:,None]).squeeze(3) + bias2

encoded.shape

(1, 2, 16)

In [31]:
x = positional_encoding(x_true)

Dmha, DweightsQ, DweightsK, DweightsV, DweightO, Dmiddle = multihead(x, heads, masked=True)

# Dmha = x
x3 = x + Dmha

Dgamma1 = np.random.randn(x.shape[2]) * 0.01
Dbeta1 = np.random.randn(1) * 0.01
x4 = norm(x3, Dgamma1, Dbeta1)

# x4 = x3

Dmha2, DweightsQ2, DweightsK2, DweightsV2, DweightO2, Dmiddle2 = multihead(x4, heads, encoded=encoded)

# Dmha2 = x4
x5 = x4 + Dmha2

Dgamma2 = np.random.randn(x.shape[2]) * 0.01
Dbeta2 = np.random.randn(1)    * 0.01
x6 = norm(x5, Dgamma2, Dbeta2)

# x6 = x5

Dweight1 = np.random.randn(x.shape[2]*4, x.shape[2]) * 0.01
Dbias1 = np.random.randn(x.shape[1],x.shape[2]*4) * 0.01
Dweight2 = np.random.randn(x.shape[2], x.shape[2]*4) * 0.01
Dbias2 = np.random.randn(x.shape[1],x.shape[2]) * 0.01

DZ1 = np.matmul(Dweight1, x6[:,:,:,None]).squeeze(3) + Dbias1
DA1 = relu(Z1)
Decoded = np.matmul(Dweight2, DA1[:,:,:,None]).squeeze(3) + Dbias2
Decoded.shape

(1, 2, 16)

In [32]:
fweight1 = np.random.randn(16, Decoded.shape[2]) * 0.01
fbias1 = np.random.randn(x.shape[1],16) * 0.01

fZ1 = np.matmul(fweight1, Decoded[:,:,:,None]).squeeze(3) + fbias1
fA1 = softmax(fZ1, False)

print(fA1.shape)

loss = mse(fA1, y_true)

(1, 2, 16)


In [33]:
def norm_back(dout, x, gamma):

    n = x.shape[0]

    x_hat = (x5-np.mean(x5, axis=(0,1)))/np.std(x5, axis=(0,1))
    std = np.std(x5, axis=(0,1))
    mean = np.mean(x5, axis=(0,1))

    dgamma = np.mean(dout * x_hat, axis=(0,1))
    dbeta = np.mean(dout, axis=(0,1,2)).reshape(1)

    dx_hat = dout * gamma

    dstd = 1/std * np.sum(dx_hat * (mean-x5), axis=(0,1))
    dmean = -1/std * np.sum(dx_hat, axis=(0,1)) + 1/(n * std) * np.sum(mean-x5, axis=(0,1))

    dx = dx_hat * 1/std + dmean * 1/n + dstd * 1/(n*std) * (x-mean)

    return dx, dgamma, dbeta

In [34]:
def fc_back(dout, x, weight, for_relu=None):
    dweight = (1/x.shape[1]) * np.matmul(np.mean(dout, axis=0).T, np.mean(x, axis=0))
    dbias = (1/x.shape[1]) * np.mean(dout, axis=0)

    dx = np.matmul(dout, weight)
    
    if for_relu is not None:
        dx = np.where(for_relu > 0, dx, 1e-8)
        dx[dx <= 0] = 1e-8
    
        return dx, dweight, dbias
    
    return dx, dweight, dbias

In [35]:
def mha_back(dout, x, weightO, weightsQ, weightsK, weightsV, middle, heads=8, masked=False, encoded=None):
    dweightO = np.matmul(np.mean(dout, axis=0).T, np.mean(middle, axis=0))
    dmiddles = np.matmul(dout, weightO)

    dweightsQ = []
    dweightsK = []
    dweightsV = []

    for i in range(heads):
        dmiddle = dmiddles[:,:,dk*i:dk*(i+1)]
        weightQ = weightsQ[i]
        weightK = weightsK[i]
        weightV = weightsV[i]

        Qk, Q, K, V, QK = attention(weightQ, weightK, weightV, x, masked=masked, encoded=encoded, back_prop=True)

        dV = np.matmul(Qk, dmiddle)
        dQk = np.matmul(V, dmiddle)

        dQK = dQk * (softmax(QK) - softmax(QK)**2)

        dK = np.matmul(dQK, Q / Q.shape[1])
        dQ = np.matmul(dQK, K / K.shape[1])

        dweightQ = np.matmul(np.mean(dQ, axis=0).T, np.mean(x, axis=0))
        dweightK = np.matmul(np.mean(dK, axis=0).T, np.mean(x, axis=0))
        dweightV = np.matmul(np.mean(dV, axis=0).T, np.mean(x, axis=0))

        dweightsQ.append(dweightQ)
        dweightsK.append(dweightK)
        dweightsV.append(dweightV)

        if i == 0:
            dx = np.matmul(dQ, weightQ)
            dencoded = np.matmul(dK, weightK)
            dencoded += np.matmul(dV, weightV)
        else:
            dx += np.matmul(dQ, weightQ)
            dencoded += np.matmul(dK, weightK)
            dencoded += np.matmul(dV, weightV)
    
    if encoded is not None:
        return dx, dweightO, dweightsQ, dweightsK, dweightsV, dencoded
    
    dx += dencoded

    return dx, dweightO, dweightsQ, dweightsK, dweightsV

In [36]:
print('loss before: ', np.mean(loss), '\n')
print('answer before: ', np.argmax(fA1, axis=(2)))
print('target:', np.argmax(y_true, axis=(2)), '\n')

for i in range(10):

    n = y_true.shape[0]

    dfA1 = -2/n * (y_true-fA1.reshape(y_true.shape))
    dfZ1 = dfA1 * (softmax(fZ1, False) - softmax(fZ1, False)**2)

    # fc
    
    dDecoded, dfweight1, dfbias1 = fc_back(dfZ1, Decoded, fweight1, for_relu=fZ1)

    # fc

    dDZ1, dDweight2, dDbias2 = fc_back(dDecoded, DA1, Dweight2)

    # fc

    dx6, dDweight1, dDbias1 = fc_back(dDZ1, x6, Dweight1)

    # norm

    dx5, dDgamma2, dDbeta2 = norm_back(dx6, x5, Dgamma2)

    # dx5 = dx6

    # mha

    dx4, dDweightO2, dDweightsQ2, dDweightsK2, dDweightsV2, Dencoded = mha_back(dx5, x4, DweightO2, DweightsQ2, DweightsK2, DweightsV2, Dmiddle2, encoded=encoded)
    # dx4 = dx5

    # skip

    dx4 = dx5*dx4 + dx5

    # norm

    dx3, dDgamma1, dDbeta1 = norm_back(dx4, x3, Dgamma1)

    # dx3 = dx4

    # mha 

    dx, dDweightO1, dDweightsQ1, dDweightsK1, dDweightsV1 = mha_back(dx3, x, DweightO, DweightsQ, DweightsK, DweightsV, Dmiddle, masked=True)

    # dx = dx3

    # skip

    dx = dx3*dx + dx3

    # fc

    dZ1, dweight2, dbias2 = fc_back(dx, A1, weight2, for_relu=Z1)

    # fc

    dx2, dweight1, dbias1 = fc_back(dZ1, x2, weight1)

    # norm

    dx1, dgamma1, dbeta1 = norm_back(dx2, x1, gamma)

    # dx1 = dx2

    # mha

    dx, dweightO, dweightsQ, dweightsK, dweightsV = mha_back(dx1, x, weightO, weightsQ, weightsK, weightsV, middle)

    dx = dx1

    learning_rate = 0.1

    weightO -= learning_rate * dweightO

    for i, j in enumerate(dweightsQ):
        weightsQ[i] -= learning_rate * j
        
    for i, j in enumerate(dweightsK):
        weightsK[i] -= learning_rate * j
        
    for i, j in enumerate(dweightsV):
        weightsV[i] -= learning_rate * j

    gamma -= learning_rate * dgamma1
    beta -= learning_rate * dbeta1

    weight1 -= learning_rate * dweight1
    bias1 -= learning_rate * dbias1
    weight2 -= learning_rate * dweight2
    bias2 -= learning_rate * dbias2

    DweightO -= learning_rate * dDweightO1

    for i, j in enumerate(dDweightsQ1):
        DweightsQ[i] -= learning_rate * j
        
    for i, j in enumerate(dDweightsK1):
        DweightsK[i] -= learning_rate * j
        
    for i, j in enumerate(dDweightsV1):
        DweightsV[i] -= learning_rate * j

    Dgamma1 -= learning_rate * dDgamma1
    Dbeta1 -= learning_rate * dDbeta1

    DweightO2 -= learning_rate * dDweightO2

    for i, j in enumerate(dDweightsQ2):
        DweightsQ[i] -= learning_rate * j
        
    for i, j in enumerate(dDweightsK2):
        DweightsK[i] -= learning_rate * j
        
    for i, j in enumerate(dDweightsV2):
        DweightsV[i] -= learning_rate * j

    Dgamma2 -= learning_rate * dDgamma2
    Dbeta2 -= learning_rate * dDbeta2

    dweight1 -= learning_rate * dDweight1
    dbias1 -= learning_rate * dDbias1
    dweight2 -= learning_rate * dDweight2
    dbias2 -= learning_rate * dDbias2

    fweight1 -= learning_rate * dfweight1
    fbias1 -= learning_rate * dfbias1


    x = positional_encoding(x_true)

    mha, middle = multihead(x, heads, weightsQ, weightsK, weightsV, weightO)
    # mha = x
    x1 = x + mha

    x2 = norm(x1, gamma, beta)

    # x2 = x1

    Z1 = np.matmul(weight1, x2[:,:,:,None]).squeeze(3) + bias1
    A1 = relu(Z1)
    encoded = np.matmul(weight2, A1[:,:,:,None]).squeeze(3) + bias2

    x = positional_encoding(x_true)

    Dmha, Dmiddle = multihead(x, heads, DweightsQ, DweightsK, DweightsV, DweightO, masked=True)
    # Dmha = x
    x3 = x + Dmha

    x4 = norm(x3, Dgamma1, Dbeta1)

    # x4 = x3

    Dmha2, Dmiddle2 = multihead(x4, heads, DweightsQ2, DweightsK2, DweightsV2, DweightO2, encoded=encoded)

    # Dmha2 = x4
    x5 = x4 + Dmha2

    x6 = norm(x5, Dgamma2, Dbeta2)

    # x6 = x5


    DZ1 = np.matmul(Dweight1, x6[:,:,:,None]).squeeze(3) + Dbias1
    DA1 = relu(Z1)
    Decoded = np.matmul(Dweight2, DA1[:,:,:,None]).squeeze(3) + Dbias2


    fZ1 = np.matmul(fweight1, Decoded[:,:,:,None]).squeeze(3) + fbias1
    fA1 = softmax(fZ1, False)

    print(np.mean(mse(fA1, y_true)))
    print()

loss before:  0.05861015427591424 

answer before:  [[ 4 14]]
target: [[0 9]] 

0.058570903577835384

0.058528723822341214

0.05848617932016602

0.05844330847726789

0.05840007064059044

0.058356484814388945

0.058312536453926506

0.05826818115021699

0.058223413761990236

0.05817822907010423



In [37]:
np.argmax(fA1, axis=(2))

array([[0, 9]], dtype=int64)

In [38]:
np.argmax(y_true, axis=(2))

array([[0, 9]], dtype=int64)

## Test of shapes

In [39]:
dweightO.shape, weightO.shape

((16, 16), (16, 16))

In [40]:
dx.shape, x.shape

((1, 2, 16), (1, 2, 16))

In [41]:
dencoded.shape, encoded.shape

NameError: name 'dencoded' is not defined

In [ ]:
dx4.shape, x4.shape

((1, 2, 16), (1, 2, 16))

In [ ]:
ddweightsQ2[0].shape, dweightsQ2[0].shape

((2, 16), (2, 16))

In [ ]:
ddweightsV2[0].shape, dweightsV2[0].shape

((2, 16), (2, 16))

In [ ]:
ddweightsK2[0].shape, dweightsK2[0].shape

((2, 16), (2, 16))

In [ ]:
ddweightO2.shape, dweightO2.shape

((16, 16), (16, 16))

In [ ]:
dx5.shape, x5.shape

((1, 2, 16), (1, 2, 16))

In [ ]:
ddbeta2.shape, dbeta2.shape

((1,), (1,))

In [ ]:
ddgamma2.shape, dgamma2.shape

((16,), (16,))

In [ ]:
ddweight1.shape, ddbias1.shape, dweight1.shape, dbias1.shape

((64, 16), (2, 64), (64, 16), (2, 64))

In [ ]:
dZ1.shape, ddZ1.shape

((1, 2, 64), (1, 2, 64))

In [ ]:
ddweight2.shape, dweight2.shape, ddbias2.shape, dbias2.shape

((16, 64), (16, 64), (2, 16), (2, 16))

In [ ]:
ddecoded.shape, dweight2.shape

((1, 2, 16), (16, 64))

In [ ]:
dfZ1.shape,fweight1.shape

((1, 2, 16), (16, 16))

In [ ]:
dfweight1.shape, fweight1.shape

((16, 16), (16, 16))